<a href="https://colab.research.google.com/github/Rifana20/fruit-veg-image-classification/blob/main/classi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install & Import
!pip install -q kaggle
from google.colab import files
files.upload()  # Upload your kaggle.json



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rifanasherin","key":"524cc4ad57a399ec549c7ee72e834094"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition
!unzip -q fruit-and-vegetable-image-recognition.zip

Dataset URL: https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition
License(s): CC0-1.0
100% 1.98G/1.98G [00:13<00:00, 252MB/s]
100% 1.98G/1.98G [00:13<00:00, 153MB/s]


In [ ]:
# 2. Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
# Fruits
fruit_classes = [
    "apple", "banana", "grapes", "kiwi", "lemon", "mango", "orange",
    "paprika", "pear", "pineapple", "pomegranate", "sweetcorn", "watermelon"
]

# Vegetables
vegetable_classes = [
    "beetroot", "bell pepper", "cabbage", "capsicum", "carrot", "cauliflower",
    "chilli pepper", "corn", "cucumber", "eggplant", "garlic", "jalepeno",
    "lettuce", "onion", "peas", "potato", "radish", "soy beans", "spinach",
    "sweetpotato", "tomato", "turnip"
]


In [ ]:
import shutil
from tqdm import tqdm

In [ ]:
# Create new folders
os.makedirs("data_binary/train/fruit", exist_ok=True)
os.makedirs("data_binary/train/vegetable", exist_ok=True)
os.makedirs("data_binary/test/fruit", exist_ok=True)
os.makedirs("data_binary/test/vegetable", exist_ok=True)

In [ ]:
# Helper function to move files into fruit/vegetable folders
def sort_images(base_path, new_path):
    for category in os.listdir(base_path):
        src_folder = os.path.join(base_path, category)
        if not os.path.isdir(src_folder):
            continue
        label = 'fruit' if category.lower() in fruit_classes else 'vegetable'
        dst_folder = os.path.join(new_path, label)

        for file in tqdm(os.listdir(src_folder), desc=f"Moving {category} to {label}"):
            src = os.path.join(src_folder, file)
            dst = os.path.join(dst_folder, file)
            shutil.copy(src, dst)

In [ ]:
# Sort train and test
sort_images("train", "data_binary/train")
sort_images("test", "data_binary/test")

Moving corn to vegetable: 100%|██████████| 10/10 [00:00<00:00, 79.15it/s]


In [ ]:
img_height, img_width = 128, 128
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    "data_binary/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_data = datagen.flow_from_directory(
    "data_binary/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 439 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, validation_data=val_data, epochs=10)

loss, accuracy = model.evaluate(val_data)
print(f"Test Accuracy: {accuracy*100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency ex

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


 3/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.4358 - loss: 1.3505

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

12/14 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9576 - loss: 0.1739

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


13/14 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step - accuracy: 0.9582 - loss: 0.1717

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


14/14 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.9596 - loss: 0.1676 - val_accuracy: 0.9512 - val_loss: 0.2177
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 574ms/step - accuracy: 0.9571 - loss: 0.1982
Test Accuracy: 95.12%


In [25]:
from tensorflow.keras.preprocessing import image
from google.colab import files

uploaded = files.upload()
for img_path in uploaded.keys():
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    label = "fruit" if prediction < 0.5 else "vegetable"
    print(f"Prediction: {label} ({prediction:.2f})")


Saving Image_1.jpg to Image_1 (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: fruit (0.30)


In [ ]:
model.save("fruit_vs_vegetable_model.h5")


In [ ]:
from google.colab import files
files.download("fruit_vs_vegetable_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>